In [2]:
import pandas as pd
import yfinance as yf

In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
ticker = yf.Ticker('2445.KL')

In [9]:
pd.DataFrame(ticker.quarterly_financials['2025-06-30'])

,2025-06-30
Tax Effect Of Unusual Items,7.331187e+05
Tax Rate For Calcs,2.615480e-01
Normalized EBITDA,9.081600e+08
Total Unusual Items,2.803000e+06
Total Unusual Items Excluding Goodwill,2.803000e+06
Net Income From Continuing Operation Net Minority Interest,3.465940e+08
Reconciled Depreciation,2.616150e+08
EBITDA,9.109630e+08
EBIT,6.493480e+08
Net Interest Income,-1.239910e+08


In [10]:
import plotly.graph_objects as go

In [11]:
labels = [
    "Total Revenue",
    "Operating Expenses",
    "Other Operating Income",
    "Operating Income",
    "Finance Costs",
    "Pretax Income",
    "Taxation",
    "Net Income"
]

In [17]:
import plotly.graph_objects as go

# --- Node labels (readable)
labels = [
    # Segment revenues
    "Plantation (derived)", "Manufacturing", "Property Development",
    # Aggregation layer
    "Total Revenue",
    # revenue split
    "Cost of Sales / Operating expenses", "Gross before Other Income",
    # other operating income flows into operating income
    "Other operating income",
    "Operating Income",
    # Non-operating adjustments (small)
    "Net non-operating items (rev/jv/impairment, net)",
    "Finance costs",
    "Profit before taxation",
    "Taxation",
    "Net profit (period)"
]

# --- Indices (for readability)
I_PLANT = labels.index("Plantation (derived)")
I_MANU  = labels.index("Manufacturing")
I_PROP  = labels.index("Property Development")
I_REV   = labels.index("Total Revenue")
I_COST  = labels.index("Cost of Sales / Operating expenses")
I_GROSS = labels.index("Gross before Other Income")
I_OTHER = labels.index("Other operating income")
I_OPINC = labels.index("Operating Income")
I_NONOP = labels.index("Net non-operating items (rev/jv/impairment, net)")
I_FC   = labels.index("Finance costs")
I_PBT  = labels.index("Profit before taxation")
I_TAX  = labels.index("Taxation")
I_NP   = labels.index("Net profit (period)")

# --- Values (RM '000) — from PDF (and derived plantation)
values_map = {
    "plantation_rev": 981_740,      # derived residual
    "manufacturing_rev": 5_384_000, # stated in report text
    "property_rev": 66_600,         # stated in report text
    "revenue_total": 6_432_340,     # statement
    "operating_expenses": 5_861_688,
    "other_oper_income": 77_470,
    "gross_before_other": 6_432_340 - 5_861_688,  # 570,652
    "operating_income": 648_122,    # statement
    "net_nonop": 1_226,             # 2,803 + 771 - 2,348
    "finance_costs": 123_991,
    "pbt": 525_357,
    "tax": 137_406,
    "net_profit": 387_951
}

# --- Links (source indices, target indices, values)
sources = [
    # segments -> Total Revenue
    I_PLANT, I_MANU, I_PROP,
    # Total Revenue -> Cost and Gross
    I_REV, I_REV,
    # Gross (remainder) -> Operating Income
    I_GROSS,
    # Other operating income -> Operating Income
    I_OTHER,
    # Operating Income -> Finance costs (expense)
    I_OPINC,
    # Operating Income -> Profit before tax (main flow)
    I_OPINC,
    # Net non-op items -> Profit before tax (small add)
    I_NONOP,
    # Profit before tax -> Tax, Net profit
    I_PBT, I_PBT
]

targets = [
    # segments -> Total Revenue
    I_REV, I_REV, I_REV,
    # Total Revenue -> Cost and Gross
    I_COST, I_GROSS,
    # Gross -> Operating Income
    I_OPINC,
    # Other operating income -> Operating Income
    I_OPINC,
    # Operating Income -> Finance costs (shows the cost being paid out of OP inc)
    I_FC,
    # Operating Income -> Profit before tax
    I_PBT,
    # Net non-op -> Profit before tax
    I_PBT,
    # Profit before tax -> Tax and Net profit
    I_TAX, I_NP
]

values = [
    # segment -> Revenue
    values_map["plantation_rev"],
    values_map["manufacturing_rev"],
    values_map["property_rev"],
    # revenue -> cost & gross
    values_map["operating_expenses"],
    values_map["gross_before_other"],
    # gross -> operating income (gross remainder becomes operating income after other income)
    values_map["gross_before_other"],
    # other operating income -> operating income
    values_map["other_oper_income"],
    # operating income -> finance costs (expense)
    values_map["finance_costs"],
    # operating income -> profit before tax (main)
    values_map["pbt"],  # note: pbt reflects op income minus finance costs plus small net non-op
    # net non-op -> profit before tax
    values_map["net_nonop"],
    # profit before tax -> tax and net profit
    values_map["tax"],
    values_map["net_profit"]
]

# Note: some flows intentionally appear twice (e.g. gross_before_other used both as
# Revenue->Gross and Gross->Operating Income) to keep the visual layering clear.

# --- Create Sankey
fig = go.Figure(data=[go.Sankey(
    arrangement="snap",
    node=dict(
        label=labels,
        pad=18,
        thickness=18,
        line=dict(color="black", width=0.5),
        color=[
            "#c7e9c0", "#7fcdbb", "#41b6c4",    # revenue segments
            "#feb24c",                          # revenue aggregate
            "#f03b20", "#fdae6b",               # cost & gross
            "#a1d99b", "#74c476",               # other income / op inc
            "#bdbdbd", "#f16913",               # non-op, finance cost
            "#2b8cbe", "#6baed6", "#9ecae1"     # pbt, tax, net profit
        ]
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        # subtle translucent links
        color=["rgba(0,0,0,0.15)"] * len(values)
    )
)])

fig.update_layout(
    title_text="KLK (2445.KL) — Q3 (3 months ended 30 Jun 2025) Financial Flow (RM'000) — Source: KLK Interim Report",
    font=dict(size=11)
)

fig.write_html("klk_sankey.html", auto_open=True)

In [18]:
import sqlite3

data = [
    ("2025Q3", "Plantation (derived)", "Total Revenue", 981740, "Residual segment revenue"),
    ("2025Q3", "Manufacturing", "Total Revenue", 5384000, "Manufacturing segment"),
    ("2025Q3", "Property Development", "Total Revenue", 66600, "Property segment"),
    ("2025Q3", "Total Revenue", "Cost of Sales / Operating expenses", 5861688, "Operating expenses (cost of sales)"),
    ("2025Q3", "Total Revenue", "Gross before Other Income", 570652, "Derived: revenue − cost"),
    ("2025Q3", "Gross before Other Income", "Operating Income", 570652, "Gross profit portion"),
    ("2025Q3", "Other operating income", "Operating Income", 77470, "Other operating income"),
    ("2025Q3", "Operating Income", "Finance costs", 123991, "Finance costs (expense)"),
    ("2025Q3", "Operating Income", "Profit before taxation", 525357, "Operating profit after finance costs"),
    ("2025Q3", "Net non-operating items (rev/jv/impairment, net)", "Profit before taxation", 1226, "Net non-operating gains/losses"),
    ("2025Q3", "Profit before taxation", "Taxation", 137406, "Tax provision"),
    ("2025Q3", "Profit before taxation", "Net profit (period)", 387951, "Net profit after tax")
]

conn = sqlite3.connect("src/data/bursa_palmai_database.db")
cur = conn.cursor()




OperationalError: unable to open database file